## EDA Project

**What sales insights can we gather from advertisement data?** We've been provided with data about used car advertisements over the course of approximately a year: information about the cars, their prices, when they were listed, and how quickly they sold.

For this project, I want to look at a small subset of the data, pickup trucks, and compare sale prices and days listed for popular makes and models.

In [146]:
import numpy as np
import pandas as pd
import plotly.express as px

vehicles = pd.read_csv('../vehicles_us.csv')

vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


### Cleaning the data

A preliminary `info()` call shows that we have some missing values in the following fields: `model_year`, `cylinders`, `odometer`, `paint_color`, and `is_4wd`. Some of these factors affect a vehicle's value, but for now, I'm mostly focused on make/model.

I want to look only at certain models, but there are some duplicates and variations in the `model` field that I should clean up first. To make it easier to find duplicates, I'll take a quick look at each automaker that offers a pickup truck. First, what makes are in the dataset?

In [94]:
def getmake(makemodel):
    '''Get the first word from a string combining make and model.'''
    make = makemodel.split(' ',1)[0]
    return make

# Go through the data getting makes. Return unique results.
print(vehicles['model'].apply(getmake).unique())

['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']


Most of these automakers have made a pickup truck at least once, but it's likely that only a few of them will be relevant to an analysis of pickup trucks in our dataset. Filtering `model` by make, I used `value_counts()` to look through short lists of models by maker.

There are no pickups in our dataset for BMW, Hyundai, Chrysler, Honda, Jeep, Subaru, Mercedes-Benz, Acura, Cadillac, Volkswagen, or Buick.

I am concerned with the following models:
- Ford: F-150, F-250, F-350, Ranger
- Toyota: Tacoma, Tundra
- Chevrolet: Silverado, Colorado
- Ram: 1500, 2500, 3500
- GMC: Sierra
- Nissan: Frontier
- Dodge: Dakota

The format of the `model` field is consistent enough that I should be able to filter pickups out of the dataset by checking the `model` field for lowercase make/model combinations that correspond to pickups. The exception is Ford, where some trucks have been entered as 'f150' and others as 'f-150`.

In [95]:
def hyphenate_trucks(model):
    '''If input string contains unhyphenated Ford model name, replace with hyphenated version'''
    model = model.replace('f150','f-150')
    model = model.replace('f250','f-250')
    model = model.replace('f350','f-350')
    return model

vehicles['model'] = vehicles['model'].apply(hyphenate_trucks)

To better limit the analysis to pickup trucks, I'll filter the `vehicles` dataframe into a `pickups`-only dataframe. I'll also add a redundant `make` column to the new dataframe, to make filtering it for graphs easier.

In [114]:
pickup_models = ['ford f-150', 'ford f-250', 'ford f-350', 'ford ranger', 'toyota tacoma', 'toyota tundra', \
                 'chevrolet colorado', 'chevrolet silverado', 'ram 1500', 'ram 2500', 'ram 3500', 'gmc sierra', \
                 'nissan frontier', 'dodge dakota']

# https://stackoverflow.com/questions/61158898/filter-pandas-where-some-columns-contain-any-of-the-words-in-a-list
# select only rows where 'model' field contains string from list of known pickup_models
pickups = vehicles[vehicles['model'].str.contains('|'.join(pickup_models))]

pickups['make'] = pickups['model'].apply(getmake)

/tmp/ipykernel_299805/3552075230.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



condition
excellent    7795
good         7209
like new     1423
fair          628
new            55
salvage        28
Name: count, dtype: int64

Let's see how the maker of a truck affects the distribution of advertised prices by plotting some histograms.

In [145]:
chevfig = px.histogram(data_frame=pickups[pickups['make'] == "chevrolet"],x='price',range_x=[0,120000],title='Price Frequencies, Chevrolet Trucks')
chevfig.update_layout(
    yaxis_title_text = 'Number of Trucks Advertised', 
    xaxis_title_text = 'Price'
    )
chevfig.show()
fordfig = px.histogram(data_frame=pickups[pickups['make'] == "ford"],x='price',range_x=[0,120000],title='Price Frequencies, Ford Trucks')
fordfig.update_layout(
    yaxis_title_text = 'Number of Trucks Advertised', 
    xaxis_title_text = 'Price'
    )
fordfig.show()

In [163]:
big3price = pickups.query("make in ['ford', 'chevrolet', 'ram']")[['price','make']]

big3price_hist = px.histogram(big3price, x="price", color="make", barmode="overlay",title="Price Frequencies by Make")
big3price_hist.update_layout(yaxis_title_text = 'Number of Trucks Advertised',xaxis_title_text = 'Price')

big3days = pickups.query("make in ['ford', 'chevrolet', 'ram']")[['days_listed','make']]
big3days_hist = px.histogram(big3days, x="days_listed", color="make", barmode="overlay",title="Listing Length Frequencies by Make")
big3days_hist.update_layout(yaxis_title_text = 'Number of Trucks Advertised',xaxis_title_text = 'Days Listed')

pickups[pickups['make'] == 'ram']['days_listed'].mean()

40.00090470446321

Okay, now let's see a scatter plot where we look at both price and number of days listed.

In [139]:
scatterfig = px.scatter(pickups, x='days_listed', y='price', color='make',hover_data=['model','condition','odometer'])
scatterfig.show()

In [143]:
pickups.query('190000 > price >= 189000 and odometer == 151248 and model == \'ford f-150\'')

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
1309,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2019-03-02,56,ford
1668,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,NaN,NaN,2019-03-20,21,ford
27375,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2018-09-25,72,ford
33434,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2019-02-05,102,ford
34389,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2019-02-02,28,ford
